## Import Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

## Hyperparameters

In [2]:
RANDOM_SEED = 171

N_FOLDS = 10
N_EPOCHS = 100
BATCH_SIZE = 64

OPTIMIZER = 'adam'
LOSS_FUNCTION = 'mean_squared_error'

HIDDEN_LAYER_UNITS = 256
OUTPUT_LAYER_UNITS = 1
HIDDEN_LAYER_ACTIVATION = 'relu'
OUTPUT_LAYER_ACTIVATION = 'sigmoid'

## Import Data

In [3]:
# Define a dictionary to store all the datasets 
season_statistics = {} 

for year in range(1994, 2024): 
    # Import the CSV file for the corresponding year as a pandas df 
    df = pd.read_csv(f"./data/{year}.csv") 
    
    # Add the dataframe to our dictionary of dataframes 
    season_statistics[year] = df

## Preprocess Data

In [4]:
# Concatenate all season data
df = pd.concat(season_statistics.values(), ignore_index=True)

# Define Categorical and Numerical Columns for Pre-Processing
selected_features = df.columns.drop(['Champion', 'Arena', 'Team', 'Year', 'G'])
# selected_features = ['W%', 'FG%', 'FGA', '3P%', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 
#                      'PPG', 'OPPG', 'Age', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'Attend./G']

# Create a Data PreProcessor
preprocessor = ColumnTransformer(transformers = [('num', StandardScaler(), selected_features)])

# Separate the Predictors and Response
X = df[selected_features]
y = df['Champion'].values

# Pre-Process the Input Data
X = preprocessor.fit_transform(X)

## Train Model

In [5]:
# Define the K-Fold Cross Validation Sets
kf = KFold(n_splits = N_FOLDS, shuffle = True, random_state = RANDOM_SEED)

# Create a List for each Model and its Accuracy
model_history, mse_history = [], []

# Perform K-Fold Cross Validation
for train_index, test_index in kf.split(X):
    
    # Split into Training and Testing Datasets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Build the Neural Network Model
    model = Sequential([
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION, input_shape=(X_train.shape[1],)),
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION),
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION),
        Dense(OUTPUT_LAYER_UNITS, activation = OUTPUT_LAYER_ACTIVATION)
    ])
    model.compile(optimizer = OPTIMIZER, loss = LOSS_FUNCTION, metrics = 'mse')
    model.fit(X_train, y_train, epochs = N_EPOCHS, batch_size = BATCH_SIZE, verbose = 1)
    
    # Predict and evaluate
    y_pred = model.predict(X_test).flatten()
    y_pred, y_test = (y_pred * 100), (y_test * 100)
    mse = mean_squared_error(y_test, y_pred)
    
    # Append the Model and MSE to the respective lists
    model_history.append(model)
    mse_history.append(mse)

Epoch 1/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0874 - mse: 0.0874
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0472 - mse: 0.0472
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0398 - mse: 0.0398
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0311 - mse: 0.0311
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0237 - mse: 0.0237
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0194 - mse: 0.0194
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0152 - mse: 0.0152
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0131 - mse: 0.0131
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0133 - mse: 0.0133
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0100 - mse: 0.0100
Epoch 11/100
13/13 [==============================] - 0s 2m

13/13 [==============================] - 0s 2ms/step - loss: 6.7831e-04 - mse: 6.7831e-04
Epoch 86/100
13/13 [==============================] - 0s 2ms/step - loss: 6.5002e-04 - mse: 6.5002e-04
Epoch 87/100
13/13 [==============================] - 0s 2ms/step - loss: 6.4510e-04 - mse: 6.4510e-04
Epoch 88/100
13/13 [==============================] - 0s 2ms/step - loss: 6.0905e-04 - mse: 6.0905e-04
Epoch 89/100
13/13 [==============================] - 0s 2ms/step - loss: 5.5693e-04 - mse: 5.5693e-04
Epoch 90/100
13/13 [==============================] - 0s 2ms/step - loss: 5.8406e-04 - mse: 5.8406e-04
Epoch 91/100
13/13 [==============================] - 0s 2ms/step - loss: 7.1549e-04 - mse: 7.1549e-04
Epoch 92/100
13/13 [==============================] - 0s 2ms/step - loss: 8.2091e-04 - mse: 8.2091e-04
Epoch 93/100
13/13 [==============================] - 0s 2ms/step - loss: 9.7645e-04 - mse: 9.7645e-04
Epoch 94/100
13/13 [==============================] - 0s 2ms/step - loss: 7.4383e-04 -

13/13 [==============================] - 0s 2ms/step - loss: 3.3429e-04 - mse: 3.3429e-04
Epoch 69/100
13/13 [==============================] - 0s 2ms/step - loss: 3.1169e-04 - mse: 3.1169e-04
Epoch 70/100
13/13 [==============================] - 0s 2ms/step - loss: 3.0111e-04 - mse: 3.0111e-04
Epoch 71/100
13/13 [==============================] - 0s 2ms/step - loss: 3.1507e-04 - mse: 3.1507e-04
Epoch 72/100
13/13 [==============================] - 0s 2ms/step - loss: 3.4297e-04 - mse: 3.4297e-04
Epoch 73/100
13/13 [==============================] - 0s 2ms/step - loss: 3.6337e-04 - mse: 3.6337e-04
Epoch 74/100
13/13 [==============================] - 0s 2ms/step - loss: 3.7457e-04 - mse: 3.7457e-04
Epoch 75/100
13/13 [==============================] - 0s 2ms/step - loss: 3.5663e-04 - mse: 3.5663e-04
Epoch 76/100
13/13 [==============================] - 0s 2ms/step - loss: 3.9287e-04 - mse: 3.9287e-04
Epoch 77/100
13/13 [==============================] - 0s 2ms/step - loss: 4.2531e-04 -

13/13 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012
Epoch 51/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 52/100
13/13 [==============================] - 0s 2ms/step - loss: 9.9417e-04 - mse: 9.9417e-04
Epoch 53/100
13/13 [==============================] - 0s 2ms/step - loss: 9.5810e-04 - mse: 9.5810e-04
Epoch 54/100
13/13 [==============================] - 0s 2ms/step - loss: 8.9515e-04 - mse: 8.9515e-04
Epoch 55/100
13/13 [==============================] - 0s 2ms/step - loss: 8.4108e-04 - mse: 8.4108e-04
Epoch 56/100
13/13 [==============================] - 0s 2ms/step - loss: 8.0453e-04 - mse: 8.0453e-04
Epoch 57/100
13/13 [==============================] - 0s 2ms/step - loss: 7.7075e-04 - mse: 7.7075e-04
Epoch 58/100
13/13 [==============================] - 0s 2ms/step - loss: 7.9367e-04 - mse: 7.9367e-04
Epoch 59/100
13/13 [==============================] - 0s 2ms/step - loss: 8.2011e-04 - mse: 8.2011

13/13 [==============================] - 0s 2ms/step - loss: 0.0017 - mse: 0.0017    
Epoch 33/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020
Epoch 34/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020
Epoch 35/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018
Epoch 36/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019
Epoch 37/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016
Epoch 38/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0015 - mse: 0.0015    
Epoch 39/100
13/13 [==============================] - 0s 1ms/step - loss: 0.0016 - mse: 0.0016    
Epoch 40/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 41/100
13/13 [==============================] - 0s 2ms/step - loss: 9.9717e-04 - mse: 9.9717e-04
Epoch 42/100
13/13 [===================

...

13/13 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016    
Epoch 44/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018
Epoch 45/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018
Epoch 46/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0015 - mse: 0.0015
Epoch 47/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013
Epoch 48/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0014 - mse: 0.0014    
Epoch 49/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 50/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013
Epoch 51/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 52/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 53/100
13/13 [=======================

13/13 [==============================] - 0s 2ms/step - loss: 0.0025 - mse: 0.0025
Epoch 26/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0025 - mse: 0.0025
Epoch 27/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0026 - mse: 0.0026
Epoch 28/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0026 - mse: 0.0026
Epoch 29/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0026 - mse: 0.0026
Epoch 30/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0021    
Epoch 31/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0023 - mse: 0.0023
Epoch 32/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0022 - mse: 0.0022
Epoch 33/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0021
Epoch 34/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020
Epoch 35/100
13/13 [==============================] - 0s 2m

13/13 [==============================] - 0s 2ms/step - loss: 0.0152 - mse: 0.0152
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0126 - mse: 0.0126
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0123 - mse: 0.0123
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0086 - mse: 0.0086
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0075 - mse: 0.0075
Epoch 11/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0064 - mse: 0.0064
Epoch 12/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049
Epoch 13/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0044 - mse: 0.0044
Epoch 14/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0033 - mse: 0.0033
Epoch 15/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0029 - mse: 0.0029
Epoch 16/100
13/13 [==============================] - 0s 2ms/step 

13/13 [==============================] - 0s 2ms/step - loss: 3.7943e-04 - mse: 3.7943e-04
Epoch 90/100
13/13 [==============================] - 0s 2ms/step - loss: 2.9286e-04 - mse: 2.9286e-04
Epoch 91/100
13/13 [==============================] - 0s 2ms/step - loss: 2.3791e-04 - mse: 2.3791e-04
Epoch 92/100
13/13 [==============================] - 0s 2ms/step - loss: 2.4939e-04 - mse: 2.4939e-04
Epoch 93/100
13/13 [==============================] - 0s 2ms/step - loss: 2.0107e-04 - mse: 2.0107e-04
Epoch 94/100
13/13 [==============================] - 0s 2ms/step - loss: 1.9272e-04 - mse: 1.9272e-04
Epoch 95/100
13/13 [==============================] - 0s 2ms/step - loss: 1.9509e-04 - mse: 1.9509e-04
Epoch 96/100
13/13 [==============================] - 0s 2ms/step - loss: 1.8905e-04 - mse: 1.8905e-04
Epoch 97/100
13/13 [==============================] - 0s 2ms/step - loss: 2.0626e-04 - mse: 2.0626e-04
Epoch 98/100
13/13 [==============================] - 0s 2ms/step - loss: 2.0964e-04 -

13/13 [==============================] - 0s 2ms/step - loss: 6.6427e-04 - mse: 6.6427e-04
Epoch 72/100
13/13 [==============================] - 0s 2ms/step - loss: 6.8561e-04 - mse: 6.8561e-04
Epoch 73/100
13/13 [==============================] - 0s 2ms/step - loss: 6.6479e-04 - mse: 6.6479e-04
Epoch 74/100
13/13 [==============================] - 0s 2ms/step - loss: 6.3549e-04 - mse: 6.3549e-04
Epoch 75/100
13/13 [==============================] - 0s 2ms/step - loss: 6.8282e-04 - mse: 6.8282e-04
Epoch 76/100
13/13 [==============================] - 0s 2ms/step - loss: 7.9910e-04 - mse: 7.9910e-04
Epoch 77/100
13/13 [==============================] - 0s 2ms/step - loss: 5.9012e-04 - mse: 5.9012e-04
Epoch 78/100
13/13 [==============================] - 0s 2ms/step - loss: 7.5807e-04 - mse: 7.5807e-04
Epoch 79/100
13/13 [==============================] - 0s 2ms/step - loss: 6.3412e-04 - mse: 6.3412e-04
Epoch 80/100
13/13 [==============================] - 0s 2ms/step - loss: 5.7983e-04 -

## Evaluate Model

In [6]:
# Print Performance Metrics
print("Average MSE:", np.mean(mse_history), '\n')
for i in range(len(model_history)):
    print(f"Model #{i + 1}, MSE: {mse_history[i]}")
    
# Get the Best Model
best_model_idx = mse_history.index(min(mse_history))
best_model = model_history[best_model_idx]

# Perform Accuracy Test
accuracy = 0
print("\n\nChampionship Prediction (Historic)\n----------------------------------\n")
for year, df in season_statistics.items():
    
    # Get the Actual Champion
    real_champion = df[df['Champion'] == 1]['Team'].values[0]
    
    # Get the Model Prediction
    X = preprocessor.fit_transform(df[selected_features])
    y = best_model.predict(X)
    pred_champion = df.iloc[np.argmax(y)]['Team']
    
    # Evaluate Results
    accuracy += int(real_champion == pred_champion)
    print(f"{year} NBA Season")
    print(f"\tActual Champion: {real_champion}")
    print(f"\tPredicted Champion: {pred_champion}")
    print()
    
print(f"Model Accuracy: {(accuracy / len(season_statistics) * 100):.2f}% ({accuracy}/{len(season_statistics)})")

Average MSE: 381.1824575140862 

Model #1, MSE: 215.19328781887285
Model #2, MSE: 342.07064326699526
Model #3, MSE: 365.67859499526867
Model #4, MSE: 271.03594424822876
Model #5, MSE: 378.96149588389125
Model #6, MSE: 483.3851483108097
Model #7, MSE: 303.84449447089753
Model #8, MSE: 460.5360999786445
Model #9, MSE: 664.4243581133993
Model #10, MSE: 326.694508053854


Championship Prediction (Historic)
----------------------------------

1/1 [==============================] - 0s 13ms/step
1994 NBA Season
	Actual Champion: Houston Rockets
	Predicted Champion: Phoenix Suns

1/1 [==============================] - 0s 14ms/step
1995 NBA Season
	Actual Champion: Houston Rockets
	Predicted Champion: Orlando Magic

1/1 [==============================] - 0s 13ms/step
1996 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [==============================] - 0s 12ms/step
1997 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [======

## Make Prediction

In [7]:
# Get the 2024 NBA Season Data
df = pd.read_csv(f"./data/2024.csv")

# Get the Model Prediction
X = preprocessor.fit_transform(df[selected_features])
y = best_model.predict(X)
pred_champion = df.iloc[np.argmax(y)]['Team']

# Evaluate Results
print(f"2024 Predicted Champion: {pred_champion}")

# Print Championship Probabilities Per Team
print("\n\nChampionship Probabilities (By Team)\n-------------------------------------")
for i in range(len(y)):
    print(f"{df.iloc[i]['Team']}: {(y[i][0] * 100):.2f}%")

1/1 [==============================] - 0s 13ms/step
2024 Predicted Champion: Boston Celtics


Championship Probabilities (By Team)
-------------------------------------
Atlanta Hawks: 0.05%
Boston Celtics: 37.21%
Brooklyn Nets: 0.35%
Charlotte Hornets: 0.00%
Chicago Bulls: 0.81%
Cleveland Cavaliers: 11.54%
Dallas Mavericks: 3.16%
Denver Nuggets: 10.67%
Detroit Pistons: 0.00%
Golden State Warriors: 6.85%
Houston Rockets: 3.32%
Indiana Pacers: 0.00%
Los Angeles Clippers: 9.51%
Los Angeles Lakers: 0.03%
Memphis Grizzlies: 0.00%
Miami Heat: 6.54%
Milwaukee Bucks: 10.32%
Minnesota Timberwolves: 31.05%
New Orleans Pelicans: 9.30%
New York Knicks: 14.29%
Oklahoma City Thunder: 0.35%
Orlando Magic: 2.70%
Philadelphia 76ers: 4.75%
Phoenix Suns: 13.79%
Portland Trail Blazers: 0.00%
Sacramento Kings: 0.05%
San Antonio Spurs: 0.00%
Toronto Raptors: 0.00%
Utah Jazz: 0.00%
Washington Wizards: 0.00%
